<a href="https://colab.research.google.com/github/LVaditya/turbo-broccoli/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snscrape

In [ ]:
!pip install streamlit

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

In [ ]:
%%writefile app.py
import streamlit as st
import snscrape.modules.twitter as sntwitter
import datetime
from datetime import date 
import pandas as pd
from tqdm import tqdm
from pymongo import MongoClient

# Streamlit part to create Interface to take inputs
st.title("Twitter Data")
st.subheader("Enter the keyword and number of tweets you want")

with st.form(key='Twitter_form'):
  
  keyword= st.text_input('Keyword')
  from_date=st.date_input("From")
  till_date=st.date_input('Till')
  query=f'{keyword} since:{from_date} until:{till_date}'
  st.write(query)
  number= st.slider('no. of tweets',min_value=0, max_value=1000, step=50)
  submit= st.form_submit_button(label='search')

if submit:
  tweet_data=[]

  for i, tweets in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):#scraping twitter data and appending it to the list

    if i > number:
      break
    tweet_data.append([tweets.date, tweets.id, tweets.content, tweets.username, tweets.url, tweets.likeCount, tweets.replyCount, tweets.retweetCount, tweets.lang])

  df= pd.DataFrame(tweet_data,columns= ['Date','Id', 'Content', 'Username', 'url', 'LikeCount', 'ReplyCount', 'RetweetCount', 'Language'])

  st.write(df)  #created a dataframe df with details and displayed 
  
  df_csv= df.to_csv(f'{keyword} .csv', index=False, encoding= 'utf-8')  #keeping a csv format of df
  df_json= df.to_json(f'{keyword} .json', orient='records')             #keeping a json format of df

  if st.button("upload"):      #to upload data to MongoDB
    client= MongoClient('mongodb+srv://adityalv:<aditya123>@cluster0.jzsqwj1.mongodb.net/?retryWrites=true&w=majority')
    db = client.twitter_scrape    #creating database twitter_scrape
    collection=db[keyword]        #creating collection of keyword as name

    df.reset_index(inplace=True)
    data_dict = df.to_dict("records")
    collection.insert_many(data_dict)

  st.download_button("download CSV",df.to_csv(),mime='text/csv')
  st.download_button("download JSON",df.to_json(),mime='application/json1')


Overwriting app.py


In [ ]:
ngrok.set_auth_token('2L6Kz7Hvawi3QB7akHGUu8CRsfi_5JPuFqERA67quEKGFuvaU')

In [ ]:
!nohup streamlit run app.py --server.port 80 &         #streamlit run using ngrok
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://9baf-35-229-63-166.ngrok.io" -> "http://localhost:80"


In [ ]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.711s


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501     #streamlit run using local tunnel